# 2. Elimination with Matrices
*(Gilbert Strang, MIT 18.06)*

## Why solve systems of linear equations?
Many real problems can be written as a system like \(Ax=b\), where we want to find the unknowns \(x\) that satisfy several equations at the same time. Solving systems helps us:
- **Predict unknown values** from known relationships (e.g., prices, forces, currents).
- **Model real situations** in engineering, physics, data fitting, and networks.
- **Compute efficiently** when problems get large, using reliable step-by-step methods.

In short: systems let us turn messy real questions into a clear math format—and elimination is a practical way to solve them.

## Task (what we will do)
Use **Gaussian elimination** to solve \(Ax=b\) and interpret each elimination step as **left-multiplication by a matrix**:
- **Elimination matrices** \(E_1, E_2, \dots\)
- **Permutation matrices** (row swaps when a pivot is zero)
- **Back substitution** after reaching an upper-triangular system \(Ux=c\)

---

## Starting point: a linear system
$$
\begin{array}{rcl}
x + 2y + z & = & 2 \\
3x + 8y + z & = & 12 \\
4y + z     & = & 2
\end{array}
$$

We will rewrite this as \(Ax=b\), apply elimination to transform \(A\) into an upper-triangular matrix \(U\), and then solve by back substitution.

---

## 1. Elimination (success case)
We solve \(Ax=b\) using Gaussian elimination.


In [3]:
import numpy as np

A = np.array([
    [1, 2, 1],
    [3, 8, 1],
    [0, 4, 1]
])

b = np.array([2, 12, 2])
print(A)
print(b)

[[1 2 1]
 [3 8 1]
 [0 4 1]]
[ 2 12  2]


### Step 1 — First Elimination (Eliminate x from Rows 2 and 3)

We start with the first pivot: **1** in Row 1.

To eliminate the value **3** below it in Row 2, we use the multiplier:

ℓ₂₁ = 3

and apply the row operation:

R₂ ← R₂ − 3R₁

Row 3 already has a zero under the pivot, so no operation is needed.


In [4]:
e1 = np.array([
    [1, 0, 0],
    [-3, 1, 0],
    [0, 0, 1]
])

A_e1 = e1 @ A
b_e1 = e1 @ b

print(A_e1)
print(b_e1)


[[ 1  2  1]
 [ 0  2 -2]
 [ 0  4  1]]
[2 6 2]


After the first elimination:
- We used the row operation: R2 <- R2 - 3R1
- Result: the x-term is eliminated from Row 2.

Step 3 — Eliminate the entry below the second pivot
- The second pivot is the entry in Row 2, Column 2: a22 = 2.
- The entry below it (Row 3, Column 2) is 4.
- Multiplier: l32 = a32 / a22 = 4 / 2 = 2

Row operation
- R3 <- R3 - 2R2

Build the elimination matrix E2
- Start with the 3x3 identity matrix.
- Put -2 in position (Row 3, Column 2) to represent "R3 <- R3 - 2R2".


In [12]:
e2 = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, -2, 1]
])

U = e2 @ A_e1
c = e2 @ b_e1

print(U)
print(c)

[[ 1  2  1]
 [ 0  2 -2]
 [ 0  0  5]]
[  2   6 -10]


We now have an upper-triangular system:

U =
[ 1  2   1 ]
[ 0  2  -2 ]
[ 0  0   5 ]

c =
[   2 ]
[   6 ]
[ -10 ]

## Step 4 — Back Substitution (solve from the bottom up)

Row 3: 5z = -10  
=> z = -2

Row 2: 2y - 2z = 6  
Substitute z = -2: 2y - 2(-2) = 6  
=> 2y + 4 = 6  
=> 2y = 2  
=> y = 1

Row 1: x + 2y + z = 2  
Substitute y = 1 and z = -2: x + 2(1) + (-2) = 2  
=> x + 0 = 2  
=> x = 2

Solution: x = 2, y = 1, z = -2


In [8]:
z = -2
y = 1
x = 2
solution_vector = np.array([x, y, z])
print(solution_vector)

[ 2  1 -2]


## Step 5 — Elimination matrices and LU factorization

We used two elimination steps:

1) Eliminate the 3 below the first pivot (Row 2 ← Row 2 − 3·Row 1)

E1 =
[ 1  0  0 ]
[ -3 1  0 ]
[ 0  0  1 ]

2) Eliminate the 4 below the second pivot (Row 3 ← Row 3 − 2·Row 2)

E2 =
[ 1  0  0 ]
[ 0  1  0 ]
[ 0 -2  1 ]

Applying elimination means multiplying on the **left** (because left multiplication performs row operations):

U = E2 · E1 · A

Undoing the elimination gives A back:

A = (E1^-1) · (E2^-1) · U

Define the lower-triangular matrix:

L = (E1^-1) · (E2^-1)

Then we get the **LU factorization**:

A = L · U


In [15]:
e1_inv = np.array([
    [1, 0, 0],
    [3, 1, 0],
    [0, 0, 1]
])

e2_inv = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 2, 1]
])

L = e1_inv @ e2_inv
print("L: \n", L)


L: 
 [[1 0 0]
 [3 1 0]
 [0 2 1]]


## Step 6 — Verify A = L U


In [16]:
AA = L @ U
print("A: \n", AA)


A: 
 [[1 2 1]
 [3 8 1]
 [0 4 1]]


## Step 7 — Elimination failure and row exchange

**Strang’s warning:** Elimination can fail if the pivot is zero (or too small).

We’ll use a tiny 2×2 example to isolate the idea.

**Matrix A (the coefficient matrix):**
A =
[ 0  2 ]
[ 1  3 ]

**Try to start elimination using the first pivot (top-left entry).**

### What goes wrong?
- First pivot = 0
- The usual elimination step needs a multiplier like:
  multiplier = (entry below pivot) / (pivot) = 1 / 0
- Division by zero → elimination fails

### Key takeaway
The system didn’t “break” — the **row order** did.  
Fix: swap the rows so a nonzero pivot comes first (that’s Step 8).


In [ ]:
A_fail = np.array([
    [0, 2],
    [1, 3]
])

print(A_fail)

## Step 8 — Permutation matrix (row exchange)

Elimination failed here not because the system is wrong, but because the pivot is in the wrong row.

The first pivot (top-left entry) is 0, so we cannot divide by it to eliminate the entry below.  
Fix: swap the rows so a nonzero pivot comes first.

Row exchange (swap Row 1 and Row 2):

R1 <-> R2

In matrix language, a row swap is done by multiplying on the left by a **permutation matrix**.


In [19]:
p = np.array([
    [0, 1],
    [1, 0]
])

PA = p @ A_fail
print(PA)

[[1 3]
 [0 2]]


## Step 9 — Left vs. right multiplication (what changes?)

### Rule (easy to remember)
- Multiply on the **left**: `P @ A`  → affects **rows** of `A`
- Multiply on the **right**: `A @ P` → affects **columns** of `A`

### What you just saw
`P @ A`

That means: **P is swapping rows** (because it is on the left).  
Result: the rows of `A` are exchanged.

### What would happen instead
`A @ P`

Now **columns** would be exchanged (because `P` is on the right).

We won’t code column swaps yet — just keep the rule in mind.

---

## Step 10 — Inverse of an elimination matrix (undoing a row step)

You already used an elimination matrix of the form:

E = [ 1  0
      -l 1 ]

Meaning: **Row 2 ← Row 2 − l · Row 1**

Strang’s key idea:
- The inverse matrix does the opposite row operation, so it **undoes** elimination.

So the inverse is:

E_inv = [ 1  0
          l  1 ]

Meaning: **Row 2 ← Row 2 + l · Row 1**

### Quick check (what you should see)
`E_inv @ E = I` (the identity matrix)


In [20]:
E = np.array([
    [1, 0],
    [-2, 1]
])

E_inv = np.array([
    [1, 0],
    [2, 1]
])

II = E @ E_inv
print(II)

[[1 0]
 [0 1]]
